# Trabalho Prático 1  #
#### André Freitas PG54707 ####
#### Bruna Macieira PG54467 ####

## Descrição e Abordagem ao Problema ##

O principal objetivo deste trabalho passa por criar um comunicação privada assíncrona em modo  “Lightweight Cryptography” entre um agente Emitter e um agente Receiver. Esta implementação em Python passa pelo uso dos módulos Cryptography, Ascon e Asyncio, cada um deles dedicado a uma determinada ação ocorrente na implementação. 

Usamos o pacote Ascon para autenticar o criptograma envolvido na counicação bem como os metadados envolventes. O Ascon é uma família de algoritmos de encriptação autenticada e de hashing concebidos para serem leves e fáceis de implementar, mesmo com contramedidas adicionais contra ataques de canais laterais. Foi concebido por uma equipa de criptógrafos da Universidade de Tecnologia de Graz, da Infineon Technologies e da Universidade de Radboud: Christoph Dobraunig, Maria Eichlseder, Florian Mendel e Martin Schläffer.

O NIST anunciou a seleção da família Ascon como o padrão de criptografia leve em fevereiro de 2023, após receber feedback público em um workshop. O NIST está trabalhando com a equipe Ascon para elaborar os padrões de criptografia leve.

Asyncio é uma biblioteca para escrever código concorrente usando a sintaxe async/await, por isso tiramos proveito dessa funcionalidade para poder implementar a comunicação cliente-servidor.

O pacote Cryptography é uma biblioteca Python que fornece fórmulas criptográficas e primitivas para programadores. O uso deste pacote serve para implementar uma AEAD com “Tweakable Block Ciphers”. 

AEAD (Authenticated Encryption with Associated Data) é um esquema de criptografia que simultaneamente assegura a confidencialidade dos dados (também conhecida como privacidade: a mensagem criptografada é impossível de entender sem o conhecimento de uma chave secreta) e a autenticidade (ou seja, é inalterável: a mensagem criptografada inclui uma etiqueta de autenticação que o remetente só pode calcular possuindo a chave secreta).

##  Primeira Parte ##

#### Imports ####

In [1]:
import asyncio
import os
from cryptography.hazmat.primitives.asymmetric import x25519
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hmac
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from secrets import token_bytes

#### Implementação ####

In [2]:
class AsconCipher:
    def __init__(self, key):
        self.key = key

    def encrypt(self, plaintext, associated_data):
        nonce = token_bytes(16)  # Generate nonce
        cipher = Cipher(algorithms.Ascon128, modes.AEAD(self.key, nonce), backend=default_backend())
        encryptor = cipher.encryptor()
        encryptor.authenticate_additional_data(associated_data)
        ciphertext = encryptor.update(plaintext) + encryptor.finalize()
        return (nonce, ciphertext, encryptor.tag)

    def decrypt(self, nonce, ciphertext, tag, associated_data):
        cipher = Cipher(algorithms.Ascon128, modes.AEAD(self.key, nonce, tag), backend=default_backend())
        decryptor = cipher.decryptor()
        decryptor.authenticate_additional_data(associated_data)
        plaintext = decryptor.update(ciphertext) + decryptor.finalize()
        return plaintext

In [3]:
async def handle_client(reader, writer):
    private_key = x25519.X25519PrivateKey.generate()
    public_key = private_key.public_key().public_bytes(serialization.Encoding.Raw, serialization.PublicFormat.Raw)

    writer.write(public_key)
    await writer.drain()

    peer_public_key_bytes = await reader.read(32)
    peer_public_key = x25519.X25519PublicKey.from_public_bytes(peer_public_key_bytes)

    shared_key = private_key.exchange(peer_public_key)

    # Derive keys using HKDF
    derived_key_material = HKDF(
        algorithm=hashes.SHA256(),
        length=64,
        salt=None,
        info=b'handshake data',
        backend=default_backend()
    ).derive(shared_key)

    cipher_key = derived_key_material[:32]
    hmac_key = derived_key_material[32:]

    cipher = AsconCipher(cipher_key)

    while True:
        nonce = await reader.readexactly(16)
        ciphertext = await reader.readuntil(separator=b'|')
        tag = await reader.readexactly(16)
        associated_data = await reader.readuntil(separator=b'||')
        plaintext = cipher.decrypt(nonce, ciphertext, tag, associated_data)

        writer.write(plaintext)
        await writer.drain()

In [4]:
async def main():
    server = await asyncio.start_server(
        handle_client, '127.0.0.1', 8888)

    addr = server.sockets[0].getsockname()
    print(f'Serving on {addr}')

    async with server:
        await server.serve_forever()

In [5]:
async def send_public_key(writer):
    private_key = x25519.X25519PrivateKey.generate()
    public_key = private_key.public_key().public_bytes(serialization.Encoding.Raw, serialization.PublicFormat.Raw)
    writer.write(public_key)
    await writer.drain()

In [6]:
async def send_message():
    while True:
        try:
            message = input("Enter message to send (or 'quit' to exit): ")
            if message.lower() == 'quit':
                break
            
            reader, writer = await asyncio.open_connection('127.0.0.1', 8888)

            # Send public key
            await send_public_key(writer)

            # Encrypt and send message
            writer.write(message.encode())
            await writer.drain()

            # Receive and decrypt response
            data = await reader.read(1024)  # Increased buffer size for receiving data
            print(f"Received message from server: {data.decode('utf-8', 'ignore')}")  # Decode using UTF-8

            writer.close()
            await writer.wait_closed()
        except KeyboardInterrupt:
            print("Exiting client...")
            break

In [11]:
task = asyncio.create_task(main())


Task exception was never retrieved
future: <Task finished name='Task-21' coro=<main() done, defined at C:\Users\andre\AppData\Local\Temp\ipykernel_12692\3117860583.py:1> exception=OSError(10048, "error while attempting to bind on address ('127.0.0.1', 8888): normalmente só é permitido uma utilização de cada endereço de socket (protocolo/endereço de rede/porta)")>
Traceback (most recent call last):
  File "C:\Users\andre\AppData\Local\Temp\ipykernel_12692\3117860583.py", line 2, in main
    server = await asyncio.start_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python312\Lib\asyncio\streams.py", line 84, in start_server
    return await loop.create_server(factory, host, port, **kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python312\Lib\asyncio\base_events.py", line 1559, in create_server
    raise OSError(err.errno, 'error while attempting '
OSError: [Errno 10048] error while attempting to bind on address ('127.0.0.1', 8888): normalm

In [12]:
await asyncio.create_task(send_message())

Received message from server: &o%1dbx힇I74


Unhandled exception in client_connected_cb
transport: <_ProactorSocketTransport fd=544>
Traceback (most recent call last):
  File "C:\Users\andre\AppData\Local\Temp\ipykernel_12692\288968504.py", line 28, in handle_client
    nonce = await reader.readexactly(16)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python312\Lib\asyncio\streams.py", line 747, in readexactly
    raise exceptions.IncompleteReadError(incomplete, n)
asyncio.exceptions.IncompleteReadError: 5 bytes read on a total of 16 expected bytes


## Segunda parte ##

#### Imports ####